In [20]:
#def munpia_title_list(search_name) , 전체 라인 한칸씩 탭하고 input()제거할 것.
import bs4
import requests
import urllib.request
from selenium import webdriver
from urllib import parse
import time
import re
from selenium.webdriver.common.keys import Keys
import pandas as pd


# 중복 제거
def duplicate(my_list):
    my_set = set(my_list) # 집합으로 변환
    new_list = list(my_set) # 리스트로 변환
    return new_list

url = "https://mm.munpia.com/main/v42/slide/pl/tab/plserial/subtab/serial/selectbox//s"
#문피아 완결 : https://mm.munpia.com/main/v42/slide/pl/tab/plserial/subtab/serial_end/selectbox//s/

driver = webdriver.Chrome("C:/chromedriver.exe")
driver.get(url)
bs_obj = bs4.BeautifulSoup(driver.page_source, "html.parser")

SCROLL_PAUSE_TIME = 2.0
title_list = []

n=1
# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")
base_xpath='//*[@id="list-pl"]/ul/li[{}]/a/div/div[2]/p[3]'
while True:
    while True:
        xpath=base_xpath.format(n)
        try:
            box_area=driver.find_element_by_xpath(xpath)
        except:
            break  
        title_list.append(box_area.text)
        n=n+1
        
        
    """ 문피아
    list_box=bs_obj.find('div',{'class','b_list_area novel-list-wrap novel-list-wrap-pl'})
    ul=list_box.find('ul')
    li_s=ul.findAll('li')

    for li in li_s:
        title=li.find('a').attrs['data-ga-label']
        title_list.append(title)
        number=li.find('a').attrs['href'].split('id=')[1]
        href="https://novel.munpia.com/"+number
        ref_list.append(href)
    """             
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

title_list = duplicate(title_list) # 중복포스트 제거

list_num = len(title_list) # 검색 결과 개수

for i in range(0, list_num):
    print("제목 : " + title_list[i] )
    
# csv 파일로 저장
data = pd.DataFrame(title_list, columns = ['제목'])
data.to_csv('munpia_title_list.csv', encoding='cp949', index=False)

제목 : 아포칼립스 2회차
제목 : 민간군사기업 BLACK ANGEL
제목 : 즐거운 아포칼립스 생활
제목 : 일루전 마켓
제목 : 나는 공산당이 싫어요!
제목 : 최상위 포식자의 아포칼립스
제목 : 아빠가 절대 갑(甲) 재벌!
제목 : 끝내주는 조상 탓
제목 : 필드 위 괴물 스트라이커
제목 : 짐승 - 복수행로
제목 : 회귀능력자 이자성
제목 : 얼굴천재로 다시 태어났다
제목 : 꿈속에서 마법을 배우는 유저
제목 : 사자의 계약
제목 : 독룡(毒龍)
제목 : 걸그룹 키우는 현질러
제목 : 칠채신기
제목 : 신화를 만드는 축구 감독
제목 : 라만차의 전사
제목 : 아저씨가 야구를 참 잘한다
제목 : 막내 공자가 제일 강해
제목 : 유물의 목소리가 들려
제목 : 알고 보니 마법 천재
제목 : 스킬빨로 축구천재
제목 : 준비된 자 이스
제목 : [新]대한도사전
제목 : 포텐셜 999,999,999의 마스터
제목 : 환생 톱스타
제목 : 마법학교 천재가 되었다
제목 : 문과라도 안 죄송한 이세계로 감
제목 : 해병(Marine)
제목 : 적국의 왕자로 사는 법
제목 : 서자의 반지
제목 : 사기당해, 지구최강!
제목 : 말단 마법공학자의 귀환
제목 : 신의 무공을 배운 초월급 헌터
제목 : 슬기로운 연예생활
제목 : 조선철강왕
제목 : 하북팽가 막내아들
제목 : 공작님, 회개해주세요!
제목 : 귀환자의 게임 라이프
제목 : 만렙이 귀환했다
제목 : 양아치가 조상을 수집함
제목 : 필드의 우리형
제목 : 글쎄, 마왕 그만뒀다니까!
제목 : 드루이드 정문
제목 : 회귀한 암살왕은 군주가 된다
제목 : 마법재벌
제목 : 정감록
제목 : 마왕 전투력 999,999,999,999
제목 : 슈퍼 솔저 최대길
제목 : 신흥무관학교 1919
제목 : 시스템 오류로 에이스 등극!
제목 : 이 악물고 레벨업!
제목 : 밑바닥부터 재벌까지
제목 : 제국의 새벽.
제목 : 제국의 소드마스터
제목 : 은퇴마왕전기
제목 : 치우의 탈
제목 : 망나니 1왕자가 되었다
제목 : 유대인과 세계